### "Simplification" attemps with OSMnx

For test (continents) cities [FUA ID]:
* (Africa) Douala [809]
* (Asia) Aleppo [1133]
* (South America) Bucaramanga [4617]
* (North America) Salt Lake City [4881]
* (Europe) Amsterdam [1427] (or Liège? - if we want a smaller city)
* (Oceania) Auckland [869]


In [1]:
# import libraries
import geopandas as gpd
import pandas as pd
import osmnx as ox
import networkx as nx

In [5]:
# read in sample metadata
sample = gpd.read_parquet("../data/sample.parquet")
sample.head(2)

,eFUA_ID,UC_num,UC_IDs,eFUA_name,Commuting,Cntry_ISO,Cntry_name,FUA_area,UC_area,FUA_p_2015,UC_p_2015,Com_p_2015,geometry,continent,iso_a3
305,9129.0,1.0,8078,Gonda,1.0,IND,India,66.0,29.0,1.074100e+06,1.066419e+06,7680.678101,"POLYGON ((81.98398 27.19657, 81.99471 27.19657...",Asia,IND
91,7578.0,6.0,10577;10581;10583;10596;10605;10607,Chongqing,1.0,CHN,China,2267.0,618.0,6.036834e+06,5.157726e+06,879107.861057,"POLYGON ((106.23972 29.52328, 106.19622 29.523...",Asia,CHN


In [7]:
# dict of fua ID : cityname
citydict = {
    809: "Douala",
    1133: "Aleppo",
    4617: "Bucaramanga",
    4881: "Salt Lake City",
    1427: "Amsterdam",
    869: "Auckland"
}

In [9]:
# read in data for example city: Amsterdam
fua = 1427
gdf = gpd.read_parquet(f"../data/{fua}/roads_osm.parquet")
gdf.head()

,u,v,key,highway,geometry
__null_dask_index__,,,,,
0,6316199,451946447,0,unclassified,"LINESTRING (628552.981 5803890.045, 628561.488..."
1,6316199,46388769,0,residential,"LINESTRING (628561.488 5803889.299, 628579.347..."
2,6316199,5825714203,0,residential,"LINESTRING (628560.362 5803877.895, 628561.488..."
3,451946447,451946445,0,unclassified,"LINESTRING (628528.888 5803891.842, 628552.981..."
4,46388769,46389218,0,residential,"LINESTRING (628579.347 5804087.372, 628583.286..."


We would need to convert it back into a MultiDiGraph in order to apply functions from the osmnx.simplification module. Therefore... let's download directly through OSMnx for now:

In [12]:
G = ox.graph_from_place(
    "Amsterdam",
    network_type = "all", # no private!
    simplify = True, # simplified!
    retain_all = False # only return the largest connected network component
    )

In [13]:
nodes, edges = ox.graph_to_gdfs(
    G, 
    nodes = True, 
    edges = True
)

TODO: find bananas with momepy here

In [14]:
ox.consolidate_intersections

networkx.classes.multidigraph.MultiDiGraph

TODO:
* identify multiedges, drop
* consolidate intersections
* simplify again (possible?)
* repeat
* try using edge bearings